In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0491, -0.0034, -0.5247, -0.2670, -0.0294, -0.1277, -0.2009,  0.2264,
         -0.2678, -0.3383,  0.0203, -0.5750, -0.5009,  0.2023, -0.1977, -0.0505,
          0.8579,  0.3745,  0.4820,  0.0084],
        [ 0.1915,  0.2687,  0.0157, -0.2554, -0.0728, -0.5460,  0.0284,  0.0630,
          0.0403, -0.2410,  0.4337, -0.6987, -0.6428, -0.0231, -0.2041, -0.0044,
          0.9947,  0.4690,  0.2575,  0.1620],
        [-0.1916,  0.3057, -0.2437, -0.3391,  0.0465, -0.4411, -0.3074,  0.0533,
          0.3032, -0.0321,  0.6502, -0.5536, -0.5228, -0.0333, -0.0780,  0.0323,
          0.9161,  0.3404,  0.2937,  0.2359]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0491, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2264, 0.0000,
         0.0000, 0.0203, 0.0000, 0.0000, 0.2023, 0.0000, 0.0000, 0.8579, 0.3745,
         0.4820, 0.0084],
        [0.1915, 0.2687, 0.0157, 0.0000, 0.0000, 0.0000, 0.0284, 0.0630, 0.0403,
         0.0000, 0.4337, 0.0000, 0.0000, 0.0000, 0.00

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0142, -0.0290, -0.0295,  ...,  0.0297,  0.0301,  0.0175],
        [-0.0174,  0.0004,  0.0135,  ...,  0.0118,  0.0039, -0.0110]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0329,  0.0048], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0119,  0.0231,  0.0332,  ..., -0.0121,  0.0255, -0.0006],
        [ 0.0178,  0.0348,  0.0124,  ..., -0.0049,  0.0420, -0.0206]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 